In [6]:
import numpy as np
import pandas as pd
from numpy.linalg import lstsq
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [8]:
# ---------------------------------------------------------------------
# 1. Data loading functions
# ---------------------------------------------------------------------


def load_etfs(path="Anonymized ETFs.csv"):
    """Load the 105 anonymised ETFs time series.

    CSV structure:
    - Row 0: 'Dates'
    - Column 'Unnamed: 0': calendar dates (DD/MM/YYYY)
    - Columns 1..105: ETF 1..ETF 105, all rebased to 100 on 01/01/2019.
    """
    df = pd.read_csv(path)
    # Drop the header row 'Dates'
    df = df.drop(index=0).reset_index(drop=True)
    df = df.rename(columns={"Unnamed: 0": "Date"})
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    df = df.set_index("Date").sort_index()
    df = df.apply(pd.to_numeric)
    return df  # (T, 105)


def load_mystery_allocation(path):
    """Load one Mystery Allocation file.

    File structure:
    Date, NAV
    01/01/2019, 100
    02/01/2019, 99.79
    ...

    We force header=None so that the first row is treated as data.
    """
    df = pd.read_csv(path, header=None, names=["Date", "NAV"])
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    df = df.set_index("Date").sort_index()
    df["NAV"] = pd.to_numeric(df["NAV"])
    return df  # (T, 1)


def load_main_asset_classes(path="Main Asset Classes.csv"):
    """Load the 14 main asset-class indices.

    CSV structure:
    - Row 2: tickers (e.g. 'SPTR500N Index')
    - Row 3: human-readable names (e.g. 'S&P 500')
    - Row 4: 'Dates'
    - Row 5+: time series

    We use row 3 as column names.
    """
    raw = pd.read_csv(path)

    names_row = raw.iloc[3].copy()
    names_row.iloc[0] = "Date"  # first column is dates

    df = raw.iloc[5:].copy()
    df.columns = names_row
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    df = df.set_index("Date").sort_index()
    df = df.apply(pd.to_numeric)

    return df  # (T, 14)

In [10]:
# ---------------------------------------------------------------------
# 2. Returns & risk/performance metrics
# ---------------------------------------------------------------------


def compute_returns(prices: pd.DataFrame, log_returns: bool = False) -> pd.DataFrame:
    """Compute daily returns from price levels.

    - If log_returns is False: simple returns P_t / P_{t-1} - 1
    - If log_returns is True:  log(P_t / P_{t-1})
    """
    if log_returns:
        rets = np.log(prices / prices.shift(1))
    else:
        rets = prices.pct_change()

    return rets.dropna(how="all")


def annualized_return(returns: pd.Series, periods_per_year: int = 252) -> float:
    """Annualised return from daily simple returns."""
    mean_daily = returns.mean()
    return (1 + mean_daily) ** periods_per_year - 1


def annualized_volatility(returns: pd.Series, periods_per_year: int = 252) -> float:
    """Annualised volatility from daily returns."""
    return returns.std(ddof=1) * np.sqrt(periods_per_year)


def sharpe_ratio(
    returns: pd.Series, rf: float = 0.0, periods_per_year: int = 252
) -> float:
    """Naïve annualised Sharpe ratio with constant risk-free rate rf.

    rf is an annual rate, e.g. rf=0.01 for 1%.
    """
    rf_daily = rf / periods_per_year
    excess = returns - rf_daily
    ann_ret = annualized_return(excess, periods_per_year)
    ann_vol = annualized_volatility(excess, periods_per_year)
    return ann_ret / ann_vol if ann_vol != 0 else np.nan


def max_drawdown(prices: pd.Series) -> float:
    """Maximum drawdown over the sample.

    Defined as min_t (P_t / max_{u<=t} P_u - 1).
    """
    running_max = prices.cummax()
    drawdown = prices / running_max - 1.0
    return drawdown.min()

In [12]:
# ---------------------------------------------------------------------
# 3. Regression utilities
# ---------------------------------------------------------------------


def regress_portfolio_on_etfs(
    port_rets: pd.Series,
    etf_rets: pd.DataFrame,
    max_etfs: int = 20,
) -> tuple[pd.Series, float]:
    """OLS regression of a portfolio on ETF returns.

    Steps:
    1) Select the max_etfs ETFs with highest absolute correlation
       with the portfolio.
    2) Run an OLS regression without intercept:
           r_port_t ≈ sum_j w_j * r_etf_j,t
    3) Return weights and R^2.

    Returns
    -------
    weights : pd.Series
        Regression coefficients (one per selected ETF).
    r2 : float
        Coefficient of determination of the regression.
    """
    # 1) Select columns (top correlations)
    corrs = etf_rets.corrwith(port_rets).abs().sort_values(ascending=False)
    cols = corrs.index[:max_etfs]
    X = etf_rets[cols].values
    y = port_rets.values

    # 2) OLS without intercept: y = X w
    w, residuals, rank, s = lstsq(X, y, rcond=None)
    weights = pd.Series(w, index=cols, name="raw_weight")

    # 3) R^2
    y_hat = X @ w
    ss_res = np.sum((y - y_hat) ** 2)
    ss_tot = np.sum((y - y.mean()) ** 2)
    r2 = 1 - ss_res / ss_tot

    return weights, r2


def normalize_long_only(weights: pd.Series) -> pd.Series:
    """Keep only positive weights and renormalise them to sum to 1."""
    w_pos = weights.clip(lower=0.0)
    total = w_pos.sum()
    if total <= 0:
        return w_pos
    return w_pos / total


def rolling_regression(
    port_rets: pd.Series,
    etf_rets: pd.DataFrame,
    window: int = 60,
    max_etfs: int = 10,
) -> pd.DataFrame:
    """Rolling OLS regression to capture time-varying weights.

    - window: length of the estimation window (in trading days);
    - max_etfs: restricts the regression to the most correlated ETFs
      over the full sample (for readability).

    Returns a DataFrame of raw regression weights with dates as index.
    """
    # Global selection of most correlated ETFs
    global_corrs = etf_rets.corrwith(port_rets).abs().sort_values(ascending=False)
    cols = global_corrs.index[:max_etfs]
    sub = etf_rets[cols]

    weights_list = []
    dates = []

    for i in range(window, len(sub)):
        X = sub.iloc[i - window : i].values
        y = port_rets.iloc[i - window : i].values
        w, *_ = lstsq(X, y, rcond=None)
        weights_list.append(w)
        dates.append(sub.index[i])

    weights_df = pd.DataFrame(weights_list, index=dates, columns=cols)
    return weights_df

In [14]:
# ---------------------------------------------------------------------
# 4. Asset-class exposure utilities
# ---------------------------------------------------------------------


def etf_asset_corr_matrix(
    etf_rets: pd.DataFrame, asset_rets: pd.DataFrame
) -> pd.DataFrame:
    """Correlation matrix between ETFs and asset classes.

    corr[i, k] = corr(ETF i, AssetClass k).
    """
    corr = pd.DataFrame(index=etf_rets.columns, columns=asset_rets.columns)
    for a in asset_rets.columns:
        corr[a] = etf_rets.corrwith(asset_rets[a])
    return corr.astype(float)


def etf_main_asset_class(
    corr_matrix: pd.DataFrame,
) -> pd.DataFrame:
    """Identify, for each ETF, its main asset class.

    For each ETF we:
    - find the asset class with highest absolute correlation;
    - record both the class name and the signed correlation.
    """
    main_class = corr_matrix.abs().idxmax(axis=1)
    corr_values = []
    for etf in corr_matrix.index:
        a = main_class.loc[etf]
        corr_values.append(corr_matrix.loc[etf, a])
    out = pd.DataFrame(
        {"main_asset_class": main_class, "corr_with_main_class": corr_values},
        index=corr_matrix.index,
    )
    return out


def portfolio_asset_exposure(
    etf_weights: pd.Series,
    corr_matrix: pd.DataFrame,
    top_k_assets: int = 3,
) -> pd.Series:
    """Approximate portfolio exposure to broad asset classes.

    For each ETF:
    - take the top_k_assets asset classes with highest |corr|;
    - split the ETF weight across these asset classes proportionally
      to |corr|;
    - sum contributions over all ETFs.

    Returns a Series indexed by asset class.
    """
    asset_classes = corr_matrix.columns
    exposure = pd.Series(0.0, index=asset_classes)

    usable_weights = etf_weights[etf_weights.index.isin(corr_matrix.index)]

    for etf, w in usable_weights.items():
        if abs(w) < 1e-6:
            continue
        etf_corrs = corr_matrix.loc[etf].dropna()
        if etf_corrs.empty:
            continue

        top = etf_corrs.abs().sort_values(ascending=False).head(top_k_assets)
        total_corr = top.abs().sum()
        if total_corr == 0:
            continue

        for asset_class, corr_val in top.items():
            exposure[asset_class] += w * abs(corr_val) / total_corr

    return exposure

In [16]:
# ---------------------------------------------------------------------
# 5. Plotting helpers
# ---------------------------------------------------------------------


def plot_mystery_nav(myst1: pd.DataFrame, myst2: pd.DataFrame):
    """Plot NAV evolution of the two Mystery Allocations."""
    plt.figure(figsize=(10, 5))
    plt.plot(myst1.index, myst1["NAV"], label="Mystery Allocation 1")
    plt.plot(myst2.index, myst2["NAV"], label="Mystery Allocation 2")
    plt.xlabel("Date")
    plt.ylabel("Net Asset Value (base 100)")
    plt.title("Evolution of Mystery Allocations")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_etf_risk_return(risk_table: pd.DataFrame):
    """Scatter plot of annualised volatility vs annualised return."""
    plt.figure(figsize=(8, 6))
    plt.scatter(risk_table["ann_vol"], risk_table["ann_ret"])
    plt.xlabel("Annualized volatility")
    plt.ylabel("Annualized return")
    plt.title("Risk–return profile of ETFs")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_vol_distribution(risk_table: pd.DataFrame):
    """Histogram of ETF volatilities."""
    plt.figure(figsize=(8, 5))
    plt.hist(risk_table["ann_vol"], bins=20)
    plt.xlabel("Annualized volatility")
    plt.ylabel("Number of ETFs")
    plt.title("Distribution of ETF volatilities")
    plt.tight_layout()
    plt.show()


def plot_asset_exposure(exposure: pd.Series, title: str):
    """Bar chart of approximate asset-class exposure."""
    plt.figure(figsize=(10, 5))
    plt.bar(exposure.index, exposure.values)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Approximate weight")
    plt.title(title)
    plt.tight_layout()
    plt.show()


def plot_rolling_weights(rolling_w: pd.DataFrame, title_prefix: str = "Mystery 2 – rolling weights"):
    """Plot rolling regression weights for the three most important ETFs."""
    avg_abs = rolling_w.abs().mean().sort_values(ascending=False)
    top_etfs = avg_abs.index[:3]

    plt.figure(figsize=(10, 5))
    for col in top_etfs:
        plt.plot(rolling_w.index, rolling_w[col], label=col)
    plt.xlabel("Date")
    plt.ylabel("Raw regression weight")
    plt.title(f"{title_prefix} (top 3 ETFs)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [18]:
# ---------------------------------------------------------------------
# 6. Main program
# ---------------------------------------------------------------------


def main():
    # -----------------------------
    # 6.1 Data loading
    # -----------------------------
    etfs = load_etfs("Anonymized ETFs.csv")
    assets = load_main_asset_classes("Main Asset Classes.csv")
    myst1 = load_mystery_allocation("Mystery Allocation 1.csv")
    myst2 = load_mystery_allocation("Mystery Allocation 2.csv")

    print("Data dimensions:")
    print(f"ETFs     : {etfs.shape}")
    print(f"Assets   : {assets.shape}")
    print(f"Mystery1 : {myst1.shape}")
    print(f"Mystery2 : {myst2.shape}")
    print()

    # -----------------------------
    # 6.2 Returns
    # -----------------------------
    rets_etfs = compute_returns(etfs)
    rets_assets = compute_returns(assets)
    rets_m1 = compute_returns(myst1)["NAV"]
    rets_m2 = compute_returns(myst2)["NAV"]

    # Align dates
    common_dates = (
        rets_etfs.index.intersection(rets_assets.index)
        .intersection(rets_m1.index)
        .intersection(rets_m2.index)
    )
    rets_etfs = rets_etfs.loc[common_dates]
    rets_assets = rets_assets.loc[common_dates]
    rets_m1 = rets_m1.loc[common_dates]
    rets_m2 = rets_m2.loc[common_dates]

    print(f"Number of common trading days: {len(common_dates)}")
    print()

    # -------------------------------------------------------------
    # 6.3 Risk & performance measures for each ETF
    # -------------------------------------------------------------
    risk_table = pd.DataFrame(index=rets_etfs.columns)
    risk_table["ann_ret"] = rets_etfs.apply(annualized_return)
    risk_table["ann_vol"] = rets_etfs.apply(annualized_volatility)
    risk_table["sharpe"] = rets_etfs.apply(sharpe_ratio)
    risk_table["max_drawdown"] = etfs.apply(max_drawdown)

    print("Sample of risk/performance measures (first 5 ETFs):")
    print(risk_table.head())
    print()

    # Classification of ETFs using k-means on (ann_ret, ann_vol, sharpe)
    features = risk_table[["ann_ret", "ann_vol", "sharpe"]].copy()
    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    kmeans = KMeans(n_clusters=4, random_state=0, n_init=10)
    labels = kmeans.fit_predict(X)
    risk_table["cluster"] = labels

    print("ETF counts by cluster:")
    print(risk_table["cluster"].value_counts().sort_index())
    print()

    print("Summary of ETF annualised volatility:")
    print(risk_table["ann_vol"].describe())
    print()

    # -------------------------------------------------------------
    # 6.4 Relationship between ETFs and asset classes
    # -------------------------------------------------------------
    corr_ea = etf_asset_corr_matrix(rets_etfs, rets_assets)
    main_class_info = etf_main_asset_class(corr_ea)

    print("Example mapping ETF -> main asset class:")
    print(main_class_info.head())
    print()

    # -------------------------------------------------------------
    # 6.5 Identification of the Mystery Allocations
    # -------------------------------------------------------------
    weights_m1_raw, r2_m1 = regress_portfolio_on_etfs(rets_m1, rets_etfs, max_etfs=20)
    weights_m2_raw, r2_m2 = regress_portfolio_on_etfs(rets_m2, rets_etfs, max_etfs=20)

    weights_m1 = normalize_long_only(weights_m1_raw)
    weights_m2 = normalize_long_only(weights_m2_raw)

    print("Mystery Allocation 1:")
    print(f"  Regression R^2: {r2_m1:.4f}")
    print("  Top 10 ETFs (long-only, normalised weights):")
    print(weights_m1.sort_values(ascending=False).head(10))
    print()

    print("Mystery Allocation 2:")
    print(f"  Regression R^2: {r2_m2:.4f}")
    print("  Top 10 ETFs (long-only, normalised weights):")
    print(weights_m2.sort_values(ascending=False).head(10))
    print()

    # Rolling regression for Mystery 2 (dynamic allocation)
    rolling_w_m2 = rolling_regression(rets_m2, rets_etfs, window=60, max_etfs=10)

    print("Preview of rolling regression weights for Mystery 2:")
    print(rolling_w_m2.head())
    print()

    # -------------------------------------------------------------
    # 6.6 Asset-class exposure of the Mystery Allocations
    # -------------------------------------------------------------
    exposure_m1 = portfolio_asset_exposure(weights_m1, corr_ea, top_k_assets=3)
    exposure_m2 = portfolio_asset_exposure(weights_m2, corr_ea, top_k_assets=3)

    print("Approximate asset-class exposure – Mystery Allocation 1:")
    print(exposure_m1.sort_values(ascending=False))
    print()

    print("Approximate asset-class exposure – Mystery Allocation 2:")
    print(exposure_m2.sort_values(ascending=False))
    print()

    # Plotting (optional; comment out if running in a non-GUI environment)
    # plot_mystery_nav(myst1, myst2)
    # plot_etf_risk_return(risk_table)
    # plot_vol_distribution(risk_table)
    # plot_asset_exposure(exposure_m1, "Asset-class exposure – Mystery Allocation 1")
    # plot_asset_exposure(exposure_m2, "Asset-class exposure – Mystery Allocation 2")
    # plot_rolling_weights(rolling_w_m2)


if __name__ == "__main__":
    main()

Data dimensions:
ETFs     : (1405, 105)
Assets   : (1668, 14)
Mystery1 : (1405, 1)
Mystery2 : (1405, 1)

Number of common trading days: 1404

Sample of risk/performance measures (first 5 ETFs):
         ann_ret   ann_vol    sharpe  max_drawdown
ETF 1   0.181709  0.203997  0.890745     -0.338309
ETF 2   0.151772  0.174450  0.870000     -0.340332
ETF 3   0.069094  0.164794  0.419275     -0.368666
ETF 4   0.138604  0.163442  0.848034     -0.337100
ETF 5   0.095882  0.168541  0.568893     -0.342183

ETF counts by cluster:
cluster
0    26
1    77
2     1
3     1
Name: count, dtype: int64

Summary of ETF annualised volatility:
count    105.000000
mean       0.150099
std        0.067524
min        0.000187
25%        0.086551
50%        0.169755
75%        0.201336
max        0.283230
Name: ann_vol, dtype: float64

Example mapping ETF -> main asset class:
       main_asset_class  corr_with_main_class
ETF 1           S&P 500              1.000000
ETF 2           S&P 500              0.970142
E